In [4]:
import time
import numpy as np
import pandas as pd
import data

In [1]:
import time
ctime = time.time()
import data_bert
ctime = time.time() - ctime
print("data_bert loading:  ", ctime)

data_bert loading:   20.78603172302246


In [2]:
ctime = time.time()
import data_bert_tree_struct
ctime = time.time() - ctime
print("data_bert_tree_struct loading:  ", ctime)

data_bert_tree_struct loading:   64.00499510765076


# Check if partition structure is correct

In [43]:
for k in range(len(data_bert_tree_struct.topic_trees_contents)):
    groups = data_bert_tree_struct.topics_group[k]["groups"]
    groups_ids = data_bert_tree_struct.topics_group[k]["group_ids"]
    # check groups no repeat
    total_len = 0
    for subgroup in range(len(data_bert_tree_struct.topics_group[k]["groups"])):
        sginfo = data_bert_tree_struct.topics_group[k]["groups"][subgroup]
        assert len(np.unique(sginfo)) == len(sginfo)
        total_len += len(sginfo)
    
    # check mutually disjoint
    assert len(np.unique(np.concatenate([subgroup for subgroup in data_bert_tree_struct.topics_group[k]["groups"]]))) == total_len
    
    # check include all
    assert len(data_bert.topics.loc[data_bert.topics["level"] >= k]) == total_len

In [62]:
for channel in range(len(data_bert_tree_struct.topics_group[0]["group_ids"])):
    channelid = data_bert_tree_struct.topics_group[0]["group_ids"][channel]
    channelstr = data_bert.topics.iloc[channelid]["channel"]
    group1 = set(data_bert.topics.index[data_bert_tree_struct.topics_group[0]["groups"][channel]])
    group2 = set(data.obtain_topics([channelstr]).index)
    assert len(group1.symmetric_difference(group2)) == 0

# Check if partition contents are correct

# Check if proximity map is correct

In [5]:
def obtain_2neighbors(node):
    total2nb = set([node.uid])
    if node.parent is not None:
        total2nb = total2nb.union(set([node.parent.uid]))
        if node.parent.parent is not None:
            total2nb = total2nb.union(set([node.parent.parent.uid]))
        total2nb = total2nb.union(set([sibling.uid for sibling in node.parent.children]))
    total2nb = total2nb.union(set([child.uid for child in node.children]))
    total2nb = total2nb.union(set([grandchild.uid for child in node.children for grandchild in child.children]))
    return total2nb

for nodeid in range(len(data_bert.topics)):
    nodestr = data_bert.topics.index[nodeid]
    node = data.topic_total_nodes[nodestr]
    nodeprox = data_bert_tree_struct.proximity_structure[nodeid]
    assert (node.parent is None) == (nodeprox["parent"] is None)
    if node.parent is not None:
        assert node.parent.uid == data_bert.topics.index[nodeprox["parent"]]
    assert len(set([child.uid for child in node.children])
        .symmetric_difference(set(data_bert.topics.index[nodeprox["children"]]))) == 0
    assert len(nodeprox["children"]) == len(set(nodeprox["children"]))
    assert len(obtain_2neighbors(node).symmetric_difference(set(data_bert.topics.index[nodeprox["close_prox"]]))) == 0
    assert len(nodeprox["close_prox"]) == len(set(nodeprox["close_prox"]))
    
    
# data_bert_tree_struct.proximity_structure

# Check if proximity correlations are correct

In [98]:
# Check train test split
assert (~data_bert.fast_contains_multi(data_bert.train_contents_num_id, data_bert_tree_struct.has_close_correlation_train_contents)).sum() == 0
assert (~data_bert.fast_contains_multi(data_bert.test_contents_num_id, data_bert_tree_struct.has_close_correlation_test_contents)).sum() == 0
assert (~data_bert.fast_contains_multi(data_bert.train_topics_num_id, data_bert_tree_struct.has_close_correlation_train_topics)).sum() == 0
assert (~data_bert.fast_contains_multi(data_bert.test_topics_num_id, data_bert_tree_struct.has_close_correlation_test_topics)).sum() == 0

In [107]:
def naive_has_correlation(content_num_id, topic_num_id):
    close_topics = data_bert_tree_struct.proximity_structure[topic_num_id]["close_prox"]
    hcorr = data_bert.has_correlations(np.array([content_num_id] * len(close_topics), dtype = np.int32), close_topics)
    return hcorr.sum() > 0

for k in range(len(data_bert_tree_struct.has_close_correlation_contents)):
    if k % 10000 == 0:
        print("Checked " + str(k) + " out of " + str(len(data_bert_tree_struct.has_close_correlation_contents)))
    assert naive_has_correlation(data_bert_tree_struct.has_close_correlation_contents[k], data_bert_tree_struct.has_close_correlation_topics[k])

Checked 0 out of 3312833
Checked 10000 out of 3312833
Checked 20000 out of 3312833
Checked 30000 out of 3312833
Checked 40000 out of 3312833
Checked 50000 out of 3312833
Checked 60000 out of 3312833
Checked 70000 out of 3312833
Checked 80000 out of 3312833
Checked 90000 out of 3312833
Checked 100000 out of 3312833
Checked 110000 out of 3312833
Checked 120000 out of 3312833
Checked 130000 out of 3312833
Checked 140000 out of 3312833
Checked 150000 out of 3312833
Checked 160000 out of 3312833
Checked 170000 out of 3312833
Checked 180000 out of 3312833
Checked 190000 out of 3312833
Checked 200000 out of 3312833
Checked 210000 out of 3312833
Checked 220000 out of 3312833
Checked 230000 out of 3312833
Checked 240000 out of 3312833
Checked 250000 out of 3312833
Checked 260000 out of 3312833
Checked 270000 out of 3312833
Checked 280000 out of 3312833
Checked 290000 out of 3312833
Checked 300000 out of 3312833
Checked 310000 out of 3312833
Checked 320000 out of 3312833
Checked 330000 out of 33

Checked 2680000 out of 3312833
Checked 2690000 out of 3312833
Checked 2700000 out of 3312833
Checked 2710000 out of 3312833
Checked 2720000 out of 3312833
Checked 2730000 out of 3312833
Checked 2740000 out of 3312833
Checked 2750000 out of 3312833
Checked 2760000 out of 3312833
Checked 2770000 out of 3312833
Checked 2780000 out of 3312833
Checked 2790000 out of 3312833
Checked 2800000 out of 3312833
Checked 2810000 out of 3312833
Checked 2820000 out of 3312833
Checked 2830000 out of 3312833
Checked 2840000 out of 3312833
Checked 2850000 out of 3312833
Checked 2860000 out of 3312833
Checked 2870000 out of 3312833
Checked 2880000 out of 3312833
Checked 2890000 out of 3312833
Checked 2900000 out of 3312833
Checked 2910000 out of 3312833
Checked 2920000 out of 3312833
Checked 2930000 out of 3312833
Checked 2940000 out of 3312833
Checked 2950000 out of 3312833
Checked 2960000 out of 3312833
Checked 2970000 out of 3312833
Checked 2980000 out of 3312833
Checked 2990000 out of 3312833
Checked 

In [118]:
import time
ctime = 0
for k in range(len(data_bert.topics)):
    close_prox = np.array(data_bert_tree_struct.proximity_structure[k]["close_prox"])
    hc_sort = np.sort(np.array(list(data_bert.topics_inv_map.loc[data_bert.correlations.index])))
    close_prox = close_prox[data_bert.fast_contains_multi(hc_sort, close_prox)]
    content_ids = list(data_bert.correlations.loc[data_bert.topics.index[close_prox], "content_ids"])
    content_ids = [ctid for content_id in content_ids for ctid in content_id.split()]
    content_num_ids = np.unique(np.array(list(data_bert.contents_inv_map.loc[content_ids])))
    assert (~data_bert_tree_struct.has_close_correlations(content_num_ids, np.repeat(k, len(content_num_ids)))).sum() == 0
    
    if k % 1000 == 0:
        print("Checked " + str(k) + " out of " + str(len(data_bert.topics)))
        ctime = time.time() - ctime
        print(ctime)
        ctime = time.time()

Checked 0 out of 76972
1676652595.767215
Checked 1000 out of 76972
23.729355812072754
Checked 2000 out of 76972
23.467053413391113
Checked 3000 out of 76972
24.38875460624695
Checked 4000 out of 76972
25.417957067489624
Checked 5000 out of 76972
24.98215365409851
Checked 6000 out of 76972
24.934101581573486
Checked 7000 out of 76972
25.571319818496704
Checked 8000 out of 76972
25.097553253173828
Checked 9000 out of 76972
28.615192890167236
Checked 10000 out of 76972
25.85480523109436
Checked 11000 out of 76972
26.13398241996765
Checked 12000 out of 76972
26.209988355636597
Checked 13000 out of 76972
25.910677194595337
Checked 14000 out of 76972
26.583041191101074
Checked 15000 out of 76972
25.198646545410156
Checked 16000 out of 76972
24.548478603363037
Checked 17000 out of 76972
26.44989252090454
Checked 18000 out of 76972
25.640844345092773
Checked 19000 out of 76972
25.655815362930298
Checked 20000 out of 76972
25.143484830856323
Checked 21000 out of 76972
26.683501482009888
Checked

# View proximity correlations statistics

In [3]:
print(len(data_bert.has_correlation_contents))
print(len(data_bert_tree_struct.has_close_correlation_contents))
print(len(data_bert_tree_struct.has_further_correlation_contents))
print(len(data_bert.topics) * len(data_bert.contents))

279919
6369102
39080895
11857305684


In [11]:
279919 / 11857305684

2.3607302321447007e-05

In [10]:
3312833 / 11857305684

0.00027939171750208545

In [12]:
6369102 / 11857305684

0.0005371458044296128

In [17]:
data_bert.correlations["content_ids"].apply(lambda x: len(x.split())).sum()

279919

In [23]:
total = 0
for channel in data_bert.topics["channel"].unique():
    cidx = data_bert.topics[(data_bert.topics["channel"] == channel) & data_bert.topics["has_content"]].index
    content_ids = [content_id for content_list in list(data_bert.correlations.loc[cidx, "content_ids"]) for content_id in content_list.split()]
    content_ids = set(content_ids)
    total += len(set(content_ids)) * len(data_bert.topics[(data_bert.topics["channel"] == channel)].index)
print(total)

337833267


In [25]:
6369102 / 337833267

0.018852796992310412

In [ ]:
6369102